In [ ]:
import pandas as pd
import tensorflow as tf

df_train_final = pd.read_csv("train.csv")

: 

In [ ]:
tf.keras.backend.clear_session()
input1=tf.keras.layers.Input(shape=(1,),name='item_id')
input6=tf.keras.layers.Input(shape=(1,),name='year')
input7=tf.keras.layers.Input(shape=(1,),name='event_name_1')
input8=tf.keras.layers.Input(shape=(1,),name='event_name_2')
input9=tf.keras.layers.Input(shape=(1,),name='season')
input10=tf.keras.layers.Input(shape=(1,23),name='Numerical_features')

emb1=tf.keras.layers.Embedding(3050,output_dim=150)(input1)
emb6=tf.keras.layers.Embedding(2017,output_dim=10)(input6)
emb7=tf.keras.layers.Embedding(32,output_dim=10)(input7)
emb8=tf.keras.layers.Embedding(6,output_dim=10)(input8)
emb9=tf.keras.layers.Embedding(5,output_dim=10)(input9)

lstm1=tf.keras.layers.LSTM(50)(emb1)
lstm6=tf.keras.layers.LSTM(10)(emb6)
lstm7=tf.keras.layers.LSTM(10)(emb7)
lstm8=tf.keras.layers.LSTM(10)(emb8)
lstm9=tf.keras.layers.LSTM(10)(emb9)
lstm10=tf.keras.layers.LSTM(10)(input10)

x1=tf.keras.layers.Flatten()(lstm1)
x6=tf.keras.layers.Flatten()(lstm6)
x7=tf.keras.layers.Flatten()(lstm7)
x8=tf.keras.layers.Flatten()(lstm8)
x9=tf.keras.layers.Flatten()(lstm9)
x10=tf.keras.layers.Flatten()(input10)

x=tf.keras.layers.Concatenate()([x1,x6,x7,x8,x9,x10])
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Dense(256,activation='sigmoid')(x)
x=tf.keras.layers.Dense(128,activation='tanh')(x)
x=tf.keras.layers.Dense(64,activation='relu')(x)
x=tf.keras.layers.Dense(1)(x)
model=tf.keras.Model([input1,input6,input7,input8,input9,input10],x)

: 

In [ ]:
model.summary()

: 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.0009),loss='mse')

In [ ]:
train = df_train_final
cv = pd.read_csv("val.csv")
train.columns

In [ ]:
import numpy as np

In [ ]:
sales_mean = train['sales'].mean()
sales_std = train['sales'].std()

y_train_scaled = (train['sales'] - sales_mean) / sales_std
y_cv_scaled = (cv['sales'] - sales_mean) / sales_std

In [ ]:
model.fit(x=[train['item_id'].values.reshape(-1,1),train['year'].values.reshape(-1,1),train['event_name_1'].values.reshape(-1,1),train['event_name_2'].values.reshape(-1,1),\
             train['season'].values.reshape(-1,1),train[[ 'roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','ewm_alpha_099_shift_28']].values.reshape(-1,1,23)],\
          y=y_train_scaled,epochs=10,verbose=1,batch_size=50000,\
          validation_data=([cv['item_id'].values.reshape(-1,1),cv['year'].values.reshape(-1,1),cv['event_name_1'].values.reshape(-1,1),cv['event_name_2'].values.reshape(-1,1),\
             cv['season'].values.reshape(-1,1),cv[[ 'roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','ewm_alpha_099_shift_28']].values.reshape(-1,1,23)],y_cv_scaled))

In [ ]:
from sklearn.metrics import mean_squared_error

def calculate_RMSE(actual, predicted):
    '''
    Calculates Root Mean Squared Error.
    Args:
        actual (numpy array): The true sales values.
        predicted (numpy array): The values predicted by your model.
    '''
    actual = actual.flatten()
    predicted = predicted.flatten()

    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    return rmse

num_cols = [
    'sell_price', 'snap_TX',
    'roll_7_shift_28_mean', 'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
    'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
    'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
    'roll_60_shift_28_std', 'roll_360_shift_28_std',
    'ewm_alpha_099_shift_28',
    'direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
    'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
    'direct_lag_84', 'direct_lag_91', 'direct_lag_98'
]


pred_log = model.predict([cv['item_id'].values.reshape(-1,1),cv['year'].values.reshape(-1,1),cv['event_name_1'].values.reshape(-1,1),cv['event_name_2'].values.reshape(-1,1),\
             cv['season'].values.reshape(-1,1),cv[[ 'roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','ewm_alpha_099_shift_28']].values.reshape(-1,1,23)])

pred_real = np.expm1(pred_log)

actual_real = cv['sales'].values

rmse_score = calculate_RMSE(actual_real, pred_real)

print("------------------------------------------------")
print(f"Final Validation RMSE (Real Scale): {rmse_score:.4f}")
print("------------------------------------------------")

In [ ]:
from sklearn.metrics import mean_squared_error

def calculate_RMSE(actual, predicted):
    '''
    Calculates Root Mean Squared Error.
    Args:
        actual (numpy array): The true sales values.
        predicted (numpy array): The values predicted by your model.
    '''
    actual = actual.flatten()
    predicted = predicted.flatten()

    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    return rmse

num_cols = [
    'sell_price', 'snap_TX',
    'roll_7_shift_28_mean', 'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
    'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
    'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
    'roll_60_shift_28_std', 'roll_360_shift_28_std',
    'ewm_alpha_099_shift_28',
    'direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
    'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
    'direct_lag_84', 'direct_lag_91', 'direct_lag_98'
]


pred_log = model.predict([cv['item_id'].values.reshape(-1,1),cv['year'].values.reshape(-1,1),cv['event_name_1'].values.reshape(-1,1),cv['event_name_2'].values.reshape(-1,1),\
             cv['season'].values.reshape(-1,1),cv[[ 'roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','ewm_alpha_099_shift_28']].values.reshape(-1,1,23)])

pred_real = np.expm1(pred_log)

actual_real = cv['sales'].values

rmse_score = calculate_RMSE(actual_real, pred_real)

print("------------------------------------------------")
print(f"Final Validation RMSE (Real Scale): {rmse_score:.4f}")
print("------------------------------------------------")